# ====================Gesture Classifier=========================

#### Importing all the required dependencies

In [61]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.optimizers import adam
import pickle
import warnings
warnings.filterwarnings("ignore")

#### Feature Engineering

In [30]:
def CreateNewFeature(df):
    for i in range(0,len(os.listdir())):
        temp = pd.read_csv(os.listdir()[i])
        acc_x_mean,acc_y_mean,acc_z_mean,gyro_x_mean,gyro_y_mean,gyro_z_mean = temp.mean()
        acc_x_max, acc_y_max, acc_z_max, gyro_x_max,gyro_y_max, gyro_z_max= temp.max()
        acc_x_min,acc_y_min,acc_z_min,gyro_x_min,gyro_y_min,gyro_z_min = temp.min()
        acc_x_mad, acc_y_mad,acc_z_mad, gyro_x_mad, gyro_y_mad, gyro_z_mad = temp.mad()
        acc_x_std,acc_y_std,acc_z_std,gyro_x_std,gyro_y_std,gyro_z_std = temp.std()
        acc_x_y_corr,_ = pearsonr(temp.X,temp.Y)
        acc_z_x_corr,_= pearsonr(temp.Y,temp.Z)
        acc_y_z_corr,_ =pearsonr(temp.Z,temp.X)
        gyro_y_z_corr,_ =pearsonr(temp['Y.1'],temp['Z.1'])
        gyro_x_y_corr,_  =pearsonr(temp['X.1'],temp['Y.1'])
        gyro_x_z_corr,_ =pearsonr(temp['X.1'],temp['Z.1'])

        asd = list([acc_x_mean, acc_y_mean, acc_z_mean, gyro_x_mean, gyro_y_mean, gyro_z_mean, acc_x_max, acc_y_max, acc_z_max, gyro_x_max,gyro_y_max, gyro_z_max, acc_x_min, acc_y_min, acc_z_min,gyro_x_min, gyro_y_min, gyro_z_min, acc_x_mad, acc_y_mad,acc_z_mad, gyro_x_mad, gyro_y_mad, gyro_z_mad, acc_x_std,acc_y_std, acc_z_std, gyro_x_std, gyro_y_std, gyro_z_std,acc_x_y_corr, acc_y_z_corr, acc_z_x_corr, gyro_x_y_corr,gyro_y_z_corr, gyro_x_z_corr])

        qwr = pd.DataFrame(asd).T
        qwr.columns = ['acc_x_mean','acc_y_mean','acc_z_mean','gyro_x_mean','gyro_y_mean','gyro_z_mean','acc_x_max','acc_y_max','acc_z_max','gyro_x_max','gyro_y_max','gyro_z_max','acc_x_min','acc_y_min','acc_z_min','gyro_x_min','gyro_y_min','gyro_z_min','acc_x_mad','acc_y_mad','acc_z_mad','gyro_x_mad','gyro_y_mad','gyro_z_mad','acc_x_std','acc_y_std','acc_z_std','gyro_x_std','gyro_y_std','gyro_z_std','acc_x_y_corr','acc_y_z_corr','acc_z_x_corr','gyro_x_y_corr','gyro_y_z_corr','gyro_x_z_corr']
        df = df.append(qwr)
    return(df)

In [26]:
### Creating empty dataframe for gestures to create new features
df = pd.DataFrame(columns=['acc_x_mean','acc_y_mean','acc_z_mean','gyro_x_mean','gyro_y_mean','gyro_z_mean','acc_x_max','acc_y_max','acc_z_max','gyro_x_max','gyro_y_max','gyro_z_max','acc_x_min','acc_y_min','acc_z_min','gyro_x_min','gyro_y_min','gyro_z_min','acc_x_mad','acc_y_mad','acc_z_mad','gyro_x_mad','gyro_y_mad','gyro_z_mad','acc_x_std','acc_y_std','acc_z_std','gyro_x_std','gyro_y_std','gyro_z_std','acc_x_y_corr','acc_y_z_corr','acc_z_x_corr','gyro_x_y_corr','gyro_y_z_corr','gyro_x_z_corr'])

In [33]:
#Directory that contains files of gesture 1
os.chdir('..\\Data\\Gesture1')
Gesture1 = CreateNewFeature(df)
Gesture1['Label'] = 1

#Directory that contains files of gesture 2
os.chdir('..\\Data\\Gesture2')
Gesture2 = CreateNewFeature(df)
Gesture2['Label'] = 0

In [39]:
#Checking the Gesture1 Data
print('Shape:',Gesture1.shape)
Gesture1.head()

Shape: (100, 37)


,acc_x_mean,acc_y_mean,acc_z_mean,gyro_x_mean,gyro_y_mean,gyro_z_mean,acc_x_max,acc_y_max,acc_z_max,gyro_x_max,...,gyro_x_std,gyro_y_std,gyro_z_std,acc_x_y_corr,acc_y_z_corr,acc_z_x_corr,gyro_x_y_corr,gyro_y_z_corr,gyro_x_z_corr,Label
0,7.639383,1.683669,0.421076,0.023885,0.067161,0.037847,14.042421,11.111830,2.396669,1.480414,...,0.384589,0.317351,2.885490,-0.934460,0.458692,-0.495601,0.143881,-0.008645,-0.482477,1
0,7.180968,1.631644,0.118934,0.032708,-0.036756,0.045715,15.644192,12.258687,1.750215,0.719887,...,0.257496,0.299725,3.297317,-0.930429,-0.271121,0.237672,0.058824,-0.501212,0.431144,1
0,7.605308,0.753712,0.407951,0.045380,0.025808,0.017605,17.109488,11.964191,4.680807,1.016767,...,0.365259,0.463217,3.768198,-0.834244,-0.101798,0.263950,-0.354192,0.056472,0.685816,1
0,7.593657,1.641047,0.284504,0.032815,0.010460,-0.019054,15.974602,11.669696,2.911438,1.231792,...,0.385553,0.355803,3.491778,-0.906493,0.254882,-0.218436,0.191172,0.646861,0.753056,1
0,7.637838,1.367001,-0.125570,0.048885,0.021598,0.026929,17.727211,12.510086,3.318465,1.387563,...,0.527415,0.379079,3.384963,-0.947903,0.070245,-0.134969,0.113766,0.081603,-0.442996,1


In [40]:
#Checking the Gesture2 Data
print('Shape:',Gesture2.shape)
Gesture2.head()

Shape: (100, 37)


,acc_x_mean,acc_y_mean,acc_z_mean,gyro_x_mean,gyro_y_mean,gyro_z_mean,acc_x_max,acc_y_max,acc_z_max,gyro_x_max,...,gyro_x_std,gyro_y_std,gyro_z_std,acc_x_y_corr,acc_y_z_corr,acc_z_x_corr,gyro_x_y_corr,gyro_y_z_corr,gyro_x_z_corr,Label
0,7.348786,-2.353464,2.170109,-0.038946,-0.053562,-0.042382,9.816526,4.307301,4.364763,1.401175,...,0.627396,0.448670,3.341099,0.858933,0.235350,0.380367,0.116562,-0.317359,0.745073,0
0,8.168369,-0.417121,1.463235,0.019863,0.013379,-0.023270,9.802161,4.051113,4.357580,0.972959,...,0.487587,0.347299,2.914826,0.819141,-0.065711,0.275725,-0.177166,-0.619076,0.684173,0
0,7.137002,-2.562758,0.725428,-0.065141,-0.068594,-0.037503,9.821315,3.802108,4.565882,0.706011,...,0.318523,0.336711,3.944875,0.756017,-0.037407,0.430939,-0.029188,-0.453647,0.787857,0
0,7.168338,-2.153814,0.786011,0.008332,0.071677,0.015989,10.192428,6.352011,4.896292,1.291219,...,0.692925,0.490629,4.259617,0.713693,-0.096938,0.257050,-0.096188,-0.555392,0.674004,0
0,6.998696,-2.818656,1.197666,-0.143105,-0.017738,-0.033353,10.010463,3.390293,3.895485,0.647978,...,0.582949,0.368426,3.974352,0.798937,0.058907,0.171220,-0.018744,-0.478529,0.692178,0


In [41]:
#Merging the data to one dataframe
Data = Gesture1.append(Gesture2)

#### Splitting the data

In [45]:
train, test = train_test_split(Data, stratify=Data["Label"], test_size = 0.25)
print('Shape of train Data:',train.shape)
print('Shape of test Data:',test.shape)

Shape of train Data: (150, 37)
Shape of test Data: (50, 37)


In [47]:
#Seperating target variable from train and test data
train_features = train.iloc[:,:36].as_matrix()
test_features = test.iloc[:,:36].as_matrix()

In [48]:
# Getting Labels of train and test data
train_results = train.Label
test_results = test.Label

In [50]:
# Changing the datatype of labels into categories
train_results = to_categorical(train_results)
test_results = to_categorical(test_results)

#### Model Architechture

In [53]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=36))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

#### Compiling the model

In [54]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#### Fitting the model on data

In [57]:
model.fit(train_features, train_results,epochs=30,batch_size=32)

Epoch 1/30
150/150 [==============================] - 0s 80us/step - loss: 0.0018 - acc: 1.0000
Epoch 2/30
150/150 [==============================] - 0s 53us/step - loss: 0.0125 - acc: 0.9900
Epoch 3/30
150/150 [==============================] - 0s 80us/step - loss: 0.0044 - acc: 0.9967
Epoch 4/30
150/150 [==============================] - 0s 53us/step - loss: 0.0027 - acc: 1.0000
Epoch 5/30
150/150 [==============================] - 0s 80us/step - loss: 0.0067 - acc: 0.9933
Epoch 6/30
150/150 [==============================] - 0s 80us/step - loss: 0.0035 - acc: 1.0000
Epoch 7/30
150/150 [==============================] - 0s 53us/step - loss: 0.0026 - acc: 1.0000
Epoch 8/30
150/150 [==============================] - 0s 80us/step - loss: 4.4334e-04 - acc: 1.0000
Epoch 9/30
150/150 [==============================] - 0s 53us/step - loss: 0.0117 - acc: 0.9933
Epoch 10/30
150/150 [==============================] - 0s 53us/step - loss: 0.0102 - acc: 0.9933
Epoch 11/30
150/150 [==============

#### Evaluating the model on test data

In [59]:
score = model.evaluate(test_features, test_results, batch_size=32)
print('Accuracy on test data:', score[1])

50/50 [==============================] - 0s 80us/step
Accuracy on test data: 1.0


#### Saving the model as a pickle file

In [62]:
pickle.dump(model,open('Gesture_Classifier_Model.pkl','wb'))